In [366]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
import keras_tuner

In [367]:
DATA_VERSION = "v2"

merged_data_path = f'../data/{DATA_VERSION}/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [368]:
# K-MEANS

# Prepare the list of unique genres
unique_genres = list(set([genre for genres in data['genres'] for genre in genres]) | set(
    [genre for genres in data['favourite_genres'] for genre in genres]))

# Convert the genres to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(unique_genres)

# Apply K-means clustering
num_clusters = 100  # Adjust this value according to your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
kmeans.fit(vectorized)
labels = kmeans.labels_

# Group the genres based on the cluster labels
clustered_genres = {}
for i, label in enumerate(labels):
    if label not in clustered_genres:
        clustered_genres[label] = []
    clustered_genres[label].append(unique_genres[i])

# Map genres to their cluster label
genre_to_cluster = {genre: label for label, genres in clustered_genres.items() for genre in genres}


# Define the function for mapping genres to simpler forms
def map_genre(genre):
    cluster_label = genre_to_cluster[genre]
    representative_genre = clustered_genres[cluster_label][
        0]  # Use the first genre in the cluster as the representative
    return representative_genre


# Save the genre_to_cluster and clustered_genres dictionaries to use for new data prediction
with open('../microservice/models/model2_genre_to_cluster.pickle', 'wb') as f:
    pickle.dump(genre_to_cluster, f)

with open('../microservice/models/model2_clustered_genres.pickle', 'wb') as f:
    pickle.dump(clustered_genres, f)

# Apply the mapping function to both 'genres' and 'favourite_genres' columns
data['genres'] = data['genres'].apply(lambda x: [map_genre(genre) for genre in x])
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: [map_genre(genre) for genre in x])

# Delete not unique genres
data['genres'] = data['genres'].apply(lambda x: list(set(x)))
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: list(set(x)))


In [369]:
# count unique genres
unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)
for genres in data['favourite_genres']:
    unique_genres.update(genres)
print(len(unique_genres))

# how many tracks are per genre
genre_count = {}
for genres in data['genres']:
    for genre in genres:
        if genre not in genre_count:
            genre_count[genre] = 0
        genre_count[genre] += 1

genre_count = sorted(genre_count.items(), key=lambda x: x[1], reverse=True)
genre_count

100


[('copla', 5319),
 ('christian rock', 4667),
 ('classic uk pop', 3360),
 ('sunshine pop', 2649),
 ('scottish folk', 1585),
 ('melodic hard rock', 1154),
 ('dark wave', 961),
 ('classic country pop', 941),
 ('canadian blues', 893),
 ('latin alternative', 640),
 ('intelligent dance music', 618),
 ('russian heavy metal', 583),
 ('czsk hip hop', 547),
 ('houston rap', 546),
 ('german singer-songwriter', 544),
 ('japanese post-hardcore', 510),
 ('british folk', 457),
 ('southern soul', 450),
 ('gothic metal', 430),
 ('progressive electro house', 418),
 ('trap argentino', 374),
 ('danish punk', 323),
 ('p funk', 319),
 ('taiwan indie', 251),
 ('australian talent show', 234),
 ('jazz organ', 207),
 ('japanese emo', 202),
 ('filter house', 199),
 ('urban contemporary', 198),
 ('disco polo', 180),
 ('piano rock', 176),
 ('vocal harmony group', 175),
 ('neo mellow', 166),
 ('chicago punk', 165),
 ('classic dutch pop', 137),
 ('reggae en espanol', 137),
 ('mariachi cristiano', 130),
 ('dancehall'

In [370]:
# balance set
skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

skipped_data (3621, 3)
not_skipped_data (6260, 3)


In [371]:
# Combine genres and favourite_genres
mlb = MultiLabelBinarizer()

data['combined_genres'] = data.apply(lambda x: x['genres'] + x['favourite_genres'], axis=1)
mlb.fit(data['combined_genres'])

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])

# save mlb to file
with open('../microservice/models/model2_mlb.pickle', 'wb') as f:
    pickle.dump(mlb, f)


In [372]:
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)

# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_train == 0))
print("Y_train skipped", np.count_nonzero(y_train == 1))
print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))

Y_train not skipped 2911
Y_train skipped 2882
Y_train skipped % 0.4974969791127222


In [373]:
mode = "train"
# mode = "tune"

# NOTATKI Z TRAINU
# 1 layer - ???
# 2 layer - 16,8 neuronow 0.0015 learning rate (50 epok, 0.648 ac) dropout (0.5)
# 2 layer - 111,41  neuronow 0.003 learning rate (20 epok, 0.670 acc) dropout (0.5)

# NOTATKI Z TUNINGU
# 2 layer - 100,100 neuronow, 0.001 learning rate (10 epok, 0.641 acc) bez dropoutu

In [374]:
def build_model(hp):
    if mode == "train":
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1])))
        model.add(Dense(units=111, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(units=41, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation="sigmoid"))

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy', metrics=['accuracy'])
    else:
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1])))

        for i in range(hp.Int("num_layers", min_value=1, max_value=3)):
            model.add(Dense(units=hp.Int(f"units_{i}", min_value=1, max_value=200, step=5), activation="relu"))
            if hp.Boolean("dropout"):
                model.add(Dropout(hp.Float("dropout_rate", min_value=0.1, max_value=0.99)))

        model.add(Dense(1, activation="sigmoid"))

        model.compile(optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=0.001, max_value=0.01, sampling="log")),loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [375]:
if mode == "train":
    model = build_model(None)
    model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=30,
        batch_size=64,
        verbose=2
    )
else:
    tuner = keras_tuner.RandomSearch(
        hypermodel=build_model,
        objective="val_accuracy",
        max_trials=50,
        executions_per_trial=1,
        overwrite=True,
        directory="tuner",
        project_name="IUM",
    )

    tuner.search_space_summary()

    tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    tuner.results_summary()

Epoch 1/30
91/91 - 2s - loss: 0.6935 - accuracy: 0.5160 - val_loss: 0.6803 - val_accuracy: 0.5825 - 2s/epoch - 19ms/step
Epoch 2/30
91/91 - 1s - loss: 0.6820 - accuracy: 0.5598 - val_loss: 0.6691 - val_accuracy: 0.5818 - 673ms/epoch - 7ms/step
Epoch 3/30
91/91 - 1s - loss: 0.6677 - accuracy: 0.5973 - val_loss: 0.6604 - val_accuracy: 0.6011 - 649ms/epoch - 7ms/step
Epoch 4/30
91/91 - 1s - loss: 0.6611 - accuracy: 0.6035 - val_loss: 0.6541 - val_accuracy: 0.6094 - 664ms/epoch - 7ms/step
Epoch 5/30
91/91 - 1s - loss: 0.6543 - accuracy: 0.6133 - val_loss: 0.6451 - val_accuracy: 0.6204 - 635ms/epoch - 7ms/step
Epoch 6/30
91/91 - 1s - loss: 0.6495 - accuracy: 0.6187 - val_loss: 0.6446 - val_accuracy: 0.6280 - 642ms/epoch - 7ms/step
Epoch 7/30
91/91 - 1s - loss: 0.6436 - accuracy: 0.6261 - val_loss: 0.6419 - val_accuracy: 0.6273 - 590ms/epoch - 6ms/step
Epoch 8/30
91/91 - 1s - loss: 0.6369 - accuracy: 0.6304 - val_loss: 0.6391 - val_accuracy: 0.6253 - 597ms/epoch - 7ms/step
Epoch 9/30
91/91 -

In [376]:
if mode == "train":
    # save model to file
    model.save('../microservice/models/model2.h5')

    print("TEST")
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_test, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
    print("Classification report:\n", classification_report(y_test, y_pred_classes))

    print("TRAIN")
    y_pred = model.predict(X_train)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_train, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
    print("Classification report:\n", classification_report(y_train, y_pred_classes))

    print("ALL")
    y_pred = model.predict(X)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y, y_pred_classes))
    print("Classification report:\n", classification_report(y, y_pred_classes))


TEST
46/46 [==============================] - 0s 2ms/step
Accuracy: 0.6487232574189096
Confusion matrix:
 [[432 278]
 [231 508]]
Classification report:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63       710
           1       0.65      0.69      0.67       739

    accuracy                           0.65      1449
   macro avg       0.65      0.65      0.65      1449
weighted avg       0.65      0.65      0.65      1449

TRAIN
182/182 [==============================] - 0s 2ms/step
Accuracy: 0.7310547212152598
Confusion matrix:
 [[1981  930]
 [ 628 2254]]
Classification report:
               precision    recall  f1-score   support

           0       0.76      0.68      0.72      2911
           1       0.71      0.78      0.74      2882

    accuracy                           0.73      5793
   macro avg       0.73      0.73      0.73      5793
weighted avg       0.73      0.73      0.73      5793

ALL
227/227 [====================